In [63]:
import os as os

import numpy as np
import pandas as pd

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# import torch
# import torch.nn as nn
# import torch.optim as optim

import tqdm

import slippi as slp

from joblib import Parallel, delayed

from multiprocessing import Manager


In [64]:
# Set the number of time steps in the model inputs
frames_per_input = 60 * 12     # 12 seconds of gameplay


def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

# Create a numpy list that is the correct size and fill it with a loop
def get_frame_data(frames, port):
    
    # sheik_inputs = np.empty((frames_per_input, 18))  # Initialize an empty NumPy array
    # for i, frame in enumerate(frames[300:  300 + frames_per_input]):   # Take frames_per_input frames. skips first 5 seconds.
        
    sheik_inputs = np.empty((len(frames)-300, 18))  # Initialize an empty NumPy array    
    for i, frame in enumerate(frames[300: ]):   # Takes all the frames. skips first 5 seconds.
        buttons = one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
        j_x = frame.ports[port].leader.pre.joystick.x
        j_y = frame.ports[port].leader.pre.joystick.y
        c_x = frame.ports[port].leader.pre.cstick.x
        c_y = frame.ports[port].leader.pre.cstick.y
        t_l = frame.ports[port].leader.pre.triggers.physical.l
        t_r = frame.ports[port].leader.pre.triggers.physical.r

        frame_data = buttons + [j_x, j_y, c_x, c_y, t_l, t_r]
        sheik_inputs[i] = frame_data

    return sheik_inputs

In [57]:
# # # Set the number of time steps in the model inputs
# # frames_per_input = 60 * 12     # 12 seconds of gameplay


# def one_hot_encode(bitmask_array):
#     labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
#     encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

#     # Create a dictionary mapping labels to their encoded values
#     label_to_value = dict(zip(labels, encoded_values))

#     # Initialize a list to store the one-hot encoded values
#     one_hot_encoded = [0] * len(labels)
#     one_hot_encoded = np.empty_like(bitmask_array)

#     # Iterate through labels and set the corresponding one-hot encoded value
#     for i,bitmask in enumerate(bitmask_array):
#                 # Iterate through labels and set the corresponding one-hot encoded value
#             for label, value in label_to_value.items():
#                 if bitmask & value:
#                     one_hot_encoded[i][labels.index(label)] = 1
                
#     return one_hot_encoded

# # Create a numpy list that is the correct size and fill it with a loop
# # Create a numpy list that is the correct size and fill it with a loop
# def get_frame_data(frames, port):
    
#     # sheik_inputs = np.empty((frames_per_input, 18))  # Initialize an empty NumPy array
#     # for i, frame in enumerate(frames[300:  300 + frames_per_input]):   # Take frames_per_input frames. skips first 5 seconds.
        
#     sheik_inputs = np.empty((len(frames)-300, 18))  # Initialize an empty NumPy array    
#     for i, frame in enumerate(frames[300: ]):   # Takes all the frames. skips first 5 seconds.
#         buttons = frame.ports[port].leader.pre.buttons.physical.value
#         j_x = frame.ports[port].leader.pre.joystick.x
#         j_y = frame.ports[port].leader.pre.joystick.y
#         c_x = frame.ports[port].leader.pre.cstick.x
#         c_y = frame.ports[port].leader.pre.cstick.y
#         t_l = frame.ports[port].leader.pre.triggers.physical.l
#         t_r = frame.ports[port].leader.pre.triggers.physical.r

#         sheik_inputs[i] = buttons + [j_x, j_y, c_x, c_y, t_l, t_r]
#         sheik_inputs[:,:12] = one_hot_encode(sheik_inputs[:,:12])

#     return sheik_inputs


In [58]:
# Function to process a single SLP file and append to shared lists
def process_slp_file(slp_file, dataset_path, time_series_list, label_list, ids):
    try:
        file_path = os.path.join(dataset_path, slp_file)
        game = slp.Game(file_path)
        frames = game.frames

        if len(frames) < 300 + frames_per_input:  # Ignore games that are <3600 frames (i.e. <60 seconds)
            return

        # List occupied ports
        occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
        port_1 = occupied_ports[0]
        port_2 = occupied_ports[1]

        if len(occupied_ports) > 2:  # Ignore games that aren't singles
            return

        # Determine characters playing
        port_1_character = game.start.players[port_1].character.name
        port_2_character = game.start.players[port_2].character.name

        frame_data = get_frame_data(frames, port_1)
        time_series_list.append(frame_data)
        label_list.append(1 if port_1_character == 'SHEIK' else 0)
        ids.append(slp_file)
        frame_data = get_frame_data(frames, port_2)
        time_series_list.append(frame_data)
        label_list.append(1 if port_2_character == 'SHEIK' else 0)
        ids.append(slp_file)
    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")

In [59]:
# Set your dataset_path and frames_per_input
dataset_path = './Slippi_Public_Dataset_v3/'
# frames_per_input = ...

slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file][:50]
print(len(slp_files))

# Create shared lists to store results
manager = Manager()
time_series_list = manager.list()
label_list = manager.list()
ids = manager.list()

# Use joblib to parallelize processing of SLP files
Parallel(n_jobs=-1, verbose=10)(delayed(process_slp_file)(slp_file, dataset_path, time_series_list, label_list, ids) for slp_file in tqdm.tqdm(slp_files))

# Create a DataFrame from the results
df = pd.DataFrame({"TimeSeries": list(time_series_list), "Label": list(label_list), "FName": list(ids)})


df.sort_values(by=['FName','Label'],inplace=True)
df.reset_index(drop=True)
print(df)

30



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
100%|██████████| 30/30 [00:00<00:00, 463.08it/s]
[Parallel(n_jobs=-1)]: Done   3 out of  30 | elapsed:    0.5s remaining:    5.2s
[Parallel(n_jobs=-1)]: Done   7 out of  30 | elapsed:    0.6s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  11 out of  30 | elapsed:    0.6s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    0.7s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.7s remaining:    0.1s


Empty DataFrame
Columns: [TimeSeries, Label, FName]
Index: []


[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.8s finished


In [67]:
# Create a numpy list that is the correct size and fill it with a loop
def get_frame_data(frames, port):
    
    # sheik_inputs = np.empty((frames_per_input, 18))  # Initialize an empty NumPy array
    # for i, frame in enumerate(frames[300:  300 + frames_per_input]):   # Take frames_per_input frames. skips first 5 seconds.
        
    sheik_inputs = np.empty((len(frames)-300, 18))  # Initialize an empty NumPy array    
    for i, frame in enumerate(frames[300: 305]):   # Takes all the frames. skips first 5 seconds.
        print(e)
        # buttons = one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
        # j_x = frame.ports[port].leader.pre.joystick.x
        # j_y = frame.ports[port].leader.pre.joystick.y
        # c_x = frame.ports[port].leader.pre.cstick.x
        # c_y = frame.ports[port].leader.pre.cstick.y
        # t_l = frame.ports[port].leader.pre.triggers.physical.l
        # t_r = frame.ports[port].leader.pre.triggers.physical.r

        # frame_data = buttons + [j_x, j_y, c_x, c_y, t_l, t_r]
        # sheik_inputs[i] = frame_data

    return sheik_inputs



file_path = os.path.join(dataset_path, slp_files[1])
game = slp.Game(file_path)
frames = game.frames


# List occupied ports
occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
port_1 = occupied_ports[0]
port_2 = occupied_ports[1]

# Determine characters playing
port_1_character = game.start.players[port_1].character.name
port_2_character = game.start.players[port_2].character.name



0
0
0
0
0
